<div class="alert alert-block alert-warning">
Planning - Acquisition - Preparation - Exploratory Analysis - <font color='red'><b>Modeling</b></font> - Product Delivery
</div>

# Decision Tree Exercises
Using the titanic data, in your classification-exercises repository, create a notebook, model.ipynb where you will do the following:

In [1]:
import pandas as pd
import numpy as np
import os
from pydataset import data

import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

from sklearn.metrics import classification_report

from env import get_db_url
from acquire import new_titanic_data
from acquire import get_titanic_data

In [2]:
# Decision Tree and Model Evaluation Imports
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix

# DECISION TREE TEMPLATE

```
_______________________________________________________________
|                              DF                             |
|-------------------|-------------------|---------------------|
|       Train       |       Validate    |          Test       |
|-------------------|-------------------|-----------|---------|
| x_train | y_train |   x_val  |  y_val |   x_test  |  y_test |
|-------------------|-------------------|-----------|---------|
```
* 1. tree_1 = DecisionTreeClassifier(max_depth = 5)
* 2. tree_1.fit(x_train, y_train)
* 3. predictions = tree_1.predict(x_train)
* 4. pd.crosstab(y_train, predictions)
* 5. val_predictions = tree_1.predict(x_val)
* 6. pd.crosstab(y_val, val_predictions)

In [3]:
titanic_df = get_titanic_data()
titanic_df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


### 1. What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [4]:
titanic_df.survived.value_counts()

0    549
1    342
Name: survived, dtype: int64

In [5]:
titanic_df['survived_baseline'] = '0'
titanic_df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone,survived_baseline
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1,0
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1,0


In [8]:
#baseline accuracy
accuracy = (titanic_df.survived == titanic_df.survived_baseline).mean()
accuracy

0.0

### 2. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

### 3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.

### 4. Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

### 5. Run through steps 2-4 using a different ```max_depth``` value.

### 6. Which model performs better on your in-sample data?

### 7. Which model performs best on your out-of-sample data, the ```validate``` set?

# EXTRA

### 1. Work through these same exercises using the Telco dataset.

### 2. Experiment with this model on other datasets with a higher number of output classes.

<div class="alert alert-block alert-warning">
Planning - Acquisition - Preparation - Exploratory Analysis - <font color='red'><b>Modeling</b></font> - Product Delivery
</div>

# Random Forest Exercises
Continue working in your model file with ```titanic``` data to do the following:

### 1. Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.

### 2. Evaluate your results using the model score, confusion matrix, and classification report.

### 3. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

### 4. Run through steps increasing your min_samples_leaf and decreasing your max_depth.

### 5. What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

### After making a few models, which one has the best performance (or closest metrics) on both train and validate?

<div class="alert alert-block alert-warning">
Planning - Acquisition - Preparation - Exploratory Analysis - <font color='red'><b>Modeling</b></font> - Product Delivery
</div>

# KNN Exercises
Continue working in your model file with the titanic dataset.

### 1. Fit a K-Nearest Neighbors classifier to your training sample and transform (i.e. make predictions on the training sample)

### 2. Evaluate your results using the model score, confusion matrix, and classification report.

### 3. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

### 4. Run through steps 2-4 setting k to 10

### 5. Run through steps 2-4 setting k to 20

### 6. What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

### 7. Which model performs best on our out-of-sample data from validate?

### Once you have completed work on the titanic dataset, try building some knn models with your telco data.

<div class="alert alert-block alert-warning">
Planning - Acquisition - Preparation - Exploratory Analysis - <font color='red'><b>Modeling</b></font> - Product Delivery
</div>

# Logistic Regression Exercises
In these exercises, we'll continue working with the titanic dataset and building logistic regression models. Throughout this exercise, be sure you are training, evaluation, and comparing models on the train and validate datasets. The test dataset should only be used for your final model.

For all of the models you create, choose a threshold that optimizes for accuracy.

Do your work for these exercises in either a notebook or a python script named ```model``` within your ```classification-exercises``` repository. ```Add```, ```commit```, and ```push``` your work.

### 1. Create a model that includes age in addition to fare and pclass. Does this model perform better than your baseline?

### 2. Include sex in your model as well. Note that you'll need to encode or create a dummy variable of this feature before including it in a model.

### 3. Try out other combinations of features and models.

### 4. Use you best 3 models to predict and evaluate on your validate sample.

### 5. Choose you best model from the validation performation, and evaluate it on the test dataset. How do the performance metrics compare to validate? to train?

## Bonus1: 
How do different strategies for handling the missing values in the age column affect model performance?

## Bonus2: 
How do different strategies for encoding sex affect model performance?

## Bonus3: 
```scikit-learn```'s ```LogisticRegression``` classifier is actually applying a regularization penalty to the coefficients (https://en.wikipedia.org/wiki/Regularized_least_squares) by default. This penalty causes the magnitude of the coefficients in the resulting model to be smaller than they otherwise would be. This value can be modified with the ```C``` hyper parameter. Small values of ```C``` correspond to a larger penalty, and large values of ```C``` correspond to a smaller penalty.
Try out the following values for ```C``` and note how the coefficients and the model's performance on both the dataset it was trained on and on the validate split are affected.
```
C = .01,.1,1,10,100,1000
```

## Bonus Bonus: 
how does scaling the data interact with your choice of ```C```?